In [1]:
import ccxt
import pandas as pd
import time
from threading import Thread


In [6]:
# instantiation of exchange
exchange = ccxt.binanceusdm({
    'apiKey': '296015e94a672f83bfb4076908741a255cd5c9f192da047d12eb63cff4c73a7e',
    'secret': '4ecd9157f7b1a8486742374e9627d4ae4cfca3b997d1532ef447dc915fad62f5',
    'enableRateLimit': True,
})
exchange.set_sandbox_mode(True)


In [19]:

class CCXTTrader():
    def __init__(self, symbol, time_frame):
        self.symbol = symbol
        self.time_frame = time_frame
        self.get_available_intervals()

    def get_available_intervals(self):
        # get available intervals
        I = []
        for key, _ in exchange.timeframes.items():
            I.append(key)
        self.available_intervals = I

    # start_trading: get historical data and start streaming live data
    def start_trading(self, start=None, Lookback=None):
        if not Lookback:
            Lookback = 1000
        if self.time_frame in self.available_intervals:
            # 1. start collecting historical data with Lookback= 1000
            self.get_history(symbol=self.symbol,
                             interval=self.time_frame, limit=Lookback)
            # 2. start streaming live data
            thread = Thread(target=self.start_klines_stream, args=(
                self.stream_candles, self.symbol, self.time_frame))
            thread.start()

    # get most recent candles
    def get_history(self, symbol, interval, start=None, limit=1000):
        if start:
            start = exchange.parse8601(start)  # convert to milliseconds

        data = exchange.fetch_ohlcv(
            symbol=symbol, timeframe=interval, since=start, limit=limit)
        last_bar_actual = data[-1][0]  # timestamp of last loaded bar
        # timestamp of current bar
        last_bar_current = exchange.fetch_ohlcv(
            symbol=symbol, timeframe=interval, limit=1)[-1][0]
        # if lastBarActual != lastBarCurrent => pull the next 1000 bars
        while last_bar_actual != last_bar_current:
            time.sleep(0.1)
            data += exchange.fetch_ohlcv(symbol=symbol, timeframe=interval,
                                         since=last_bar_actual, limit=limit)[1:]
            last_bar_actual = data[-1][0]
            last_bar_current = exchange.fetch_ohlcv(
                symbol=symbol, timeframe=interval, limit=1)[-1][0]

        df = pd.DataFrame(data)
        df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
        df.Date = pd.to_datetime(df.Date, unit='ms')
        df.set_index('Date', inplace=True)
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        self.last_bar_time = df.index[-1]  # time@last_bar

        self.data = df

    # define streaming function
    def start_klines_stream(self, callback, symbol, interval):
        # global RUNNING  # global variable to control the loop externally
        self.running = True

        while self.running:
            res = exchange.fetch_ohlcv(
                symbol=symbol, timeframe=interval, limit=2)
            if len(res) == 0:
                print('no data')
            else:
                callback(res)
            time.sleep(1)

    # streaming controller
    def stop_stream(self):
        self.running = False

    # callback to handle streamed candles
    def stream_candles(self, res):
        # define how to process the streamed data

        # extract data from the response
        start_time = pd.to_datetime(res[-1][0], unit='ms')
        first = res[-1][1]
        high = res[-1][2]
        low = res[-1][3]
        close = res[-1][4]
        volume = res[-1][5]
        
        print(res) # test

        # check if a bar is complete
        if start_time == self.last_bar_time:
            complete = False
        else:  # a new bar is created => add the first bar # res[0]
            complete = True
            print(res[0]) # test
            if len(res) == 2:
                self.data.loc[self.last_bar_time] = [res[0][1],
                                                     res[0][2], res[0][3], res[0][4], res[0][5], True]
            else:
                self.data.loc[self.last_bar_time, 'Complete'] = complete
            self.last_bar_time = start_time  # update the last_bar_time

        # print sth
        print('.', end='', flush=True)

        # feed self.data<df> with the latest completed bar
        self.data.loc[start_time] = [first, high, low, close, volume, complete, False]        

        # if a new bar is complete, do trading according to defined strategy
        if complete:
            print("\n", "Define Strategy and execute Trades")
            # TODO: define strategy and execute trades
            # self.define_strategy()
            # self.execute_trades()


In [12]:
trader = CCXTTrader(symbol='ETHUSDT', time_frame='1m')

In [20]:
trader.start_trading(start= None, Lookback= 10)

[[1679392380000, 1725.85, 1730.0, 1725.85, 1730.0, 47.928], [1679392440000, 1725.85, 1725.85, 1725.85, 1725.85, 4.295]]
.[[1679392380000, 1725.85, 1730.0, 1725.85, 1730.0, 47.928], [1679392440000, 1725.85, 1725.85, 1725.85, 1725.85, 4.295]]
.[[1679392380000, 1725.85, 1730.0, 1725.85, 1730.0, 47.928], [1679392440000, 1725.85, 1725.85, 1725.85, 1725.85, 4.295]]
.[[1679392380000, 1725.85, 1730.0, 1725.85, 1730.0, 47.928], [1679392440000, 1725.85, 1725.85, 1725.85, 1725.85, 4.295]]
.[[1679392380000, 1725.85, 1730.0, 1725.85, 1730.0, 47.928], [1679392440000, 1725.85, 1725.85, 1725.85, 1725.85, 4.295]]
.[[1679392380000, 1725.85, 1730.0, 1725.85, 1730.0, 47.928], [1679392440000, 1725.85, 1725.85, 1725.85, 1725.85, 4.295]]
.[[1679392380000, 1725.85, 1730.0, 1725.85, 1730.0, 47.928], [1679392440000, 1725.85, 1725.85, 1725.85, 1725.85, 4.295]]
.[[1679392380000, 1725.85, 1730.0, 1725.85, 1730.0, 47.928], [1679392440000, 1725.85, 1725.85, 1725.85, 1725.85, 4.295]]
.[[1679392380000, 1725.85, 1730.0

In [21]:
trader.stop_stream()

In [22]:
print(trader.data)

                        Open     High      Low    Close   Volume  Complete
Date                                                                      
2023-03-21 09:45:00  1718.00  1725.85  1718.00  1719.00   56.384      True
2023-03-21 09:46:00  1719.00  1725.85  1719.00  1719.00    3.196      True
2023-03-21 09:47:00  1719.00  1730.00  1719.00  1725.85   54.583      True
2023-03-21 09:48:00  1730.00  1730.00  1725.85  1725.85    5.831      True
2023-03-21 09:49:00  1725.85  1730.00  1725.85  1730.00  147.990      True
2023-03-21 09:50:00  1730.00  1730.00  1725.85  1730.00   11.587      True
2023-03-21 09:51:00  1725.85  1730.00  1725.85  1725.85   14.476      True
2023-03-21 09:52:00  1730.00  1730.00  1725.85  1730.00   14.880      True
2023-03-21 09:53:00  1725.85  1730.00  1725.85  1730.00   47.928      True
2023-03-21 09:54:00  1725.85  1730.00  1725.85  1725.85   25.219      True
